In [3]:
from twikit import Client
import time
import json
from twikit import TwitterException 
import pandas as pd
import sqlite3

# this API requires authentication
f = open('authentication.txt', 'r')
auth = f.read()
f.close()
auth_token = auth.split("\n")

# don't hardcode your email and password into something!!!
# the auth is in gitignore so I won't get hacked
USERNAME = str(auth_token[0])
EMAIL = str(auth_token[1])
PASSWORD = str(auth_token[2])

# Initialize client
client = Client(language='en-US', http2=True)

# Login to the service with provided user credentials
client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

C:\Users\eaesp\AppData\Local\Temp\ipykernel_30016\4121500996.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


{'flow_token': 'g;171374186789060094:-1713741867950:sYnirvlQKa5avXNDiDKRXWoF:14',
 'status': 'success',
 'subtasks': [{'subtask_id': 'LoginSuccessSubtask',
   'open_account': {'user': {'id': 1547081484695216130,
     'id_str': '1547081484695216130',
     'name': 'Eloragh Espie',
     'screen_name': 'EloraghEspie'},
    'next_link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'SuccessExit'},
    'attribution_event': 'login'}},
  {'subtask_id': 'SuccessExit',
   'open_link': {'link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'LoginOpenHomeTimeline'}}},
  {'subtask_id': 'LoginOpenHomeTimeline',
   'open_home_timeline': {'next_link': {'link_type': 'abort',
     'link_id': 'next_link'}}}]}

In [2]:
# Twitter LOVES to ban people when they log in repeatedly
# saving the cookies makes sure I don't get banned (often)

client.get_cookies()
client.save_cookies('cookies.json')
with open('cookies.json', 'r', encoding='UTF8') as f:
    client.set_cookies(json.load(f))

In [3]:
from twikit import UserUnavailable
from twikit import UserNotFound

def get_user_id(num, handle):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('cookies.json')
    
    # pull IDs and insert them back into the list
    try:
        user_id = client.get_user_by_screen_name(handle).id
        print(f'{num} Success {handle}, {user_id}')

    # if user is inaccessible these will keep the loop from breaking
    except UserUnavailable:
        print(f"{num} User {handle} unavailable")
    except UserNotFound:
        print(f'{num} User {handle} not found')

    return(user_id)

In [50]:
def create_key(user_id, year):
    # load the cookies so you don't login a million times and get banned
    return(int(str(user_id) + str(year)))

In [78]:
from math import ceil
import requests
from twikit import TooManyRequests
from twikit.utils import Endpoint

# this is a housekeeping function
# twitter API can throw rate limits
# they're kind of like timeouts
# this function just shows me how much longer I will be in timeout for

def get_limit_reset_time(endpoint: str):
    res = requests.get(
        endpoint,
        headers=client._base_headers,
        cookies=client.get_cookies()
    )
    return ceil(int(res.headers['x-rate-limit-reset']) - time.time())

In [108]:
# timeout check for accessing user IDs
try:
    print(client.get_user_by_screen_name('BarackObama'))
except TooManyRequests:
    
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

<User id="813286">


In [23]:
data = pd.read_csv("data/twitter politicians.csv", keep_default_na=False)
df = pd.DataFrame(data)

# turn pandas into list
# there is probably a better way to do this but this works 
data_list = []
for index, row in df.iterrows():

    row_list = df.loc[index, :].values.flatten().tolist()
    data_list.append(row_list)
    
batch1 = [item for item in data_list[0:77]]
batch2 = [item for item in data_list[77:]]

mass_list = [batch1, batch2]

[['', 'Barack Obama', 'BarackObama'], ['', 'Joe Biden', 'JoeBiden'], ['', 'Dennis Kucinich', 'Dennis_Kucinich'], ['', 'Bill Richardson', 'GovRichardson'], ['', 'Mike Huckabee', 'GovMikeHuckabee'], ['', 'Hillary Clinton', 'HillaryClinton'], ['', 'Ron Paul', 'RonPaul'], ['', 'John McCain', 'SenJohnMcCain'], ['', 'Newt Gingrich', 'newtgingrich'], ['', 'Mike Gravel', 'MikeGravel_US'], ['', 'Alan Keyes', 'loyaltoliberty'], ['', 'Mitt Romney', 'MittRomney'], ['', 'Fred Thompson', 'fredthompson'], ['', 'Chris Dodd', 'SenChrisDodd'], ['', 'Rudy Guiliani', 'RudyGiuliani'], ['', 'Ralph Nader', 'RalphNader'], ['', 'Tom Tancredo', 'ttancredo'], ['', 'Jill Stein', 'DrJillStein'], ['', 'Virgil Goode', 'VirgilGoode'], ['', 'Gary Johnson', 'GovGaryJohnson'], ['', 'Rick Santorum', 'RickSantorum'], ['', 'Hllary Clinton', 'HillaryClinton'], ['', 'Bernie Sanders', 'SenSanders'], ['', 'Donald Trump', 'realDonaldTrump'], ['', 'Jeb Bush', 'JebBush'], ['', 'Ted Cruz', 'TedCruz'], ['', 'Marco Rubio', 'MarcoRub

In [25]:
i = 1
final_list = []

for lst in mass_list:
    # tracking which batch we are on
    print(i)
    j = 1
    for item in lst:
        user_id = get_user_id(j, item[2])
        item[0] = user_id
        final_list.append(item)
        time.sleep(1)
        j += 1

    if i == 1:
        time.sleep(900) # cooldown to avoid rate limits

    i += 1
    print("") 

1
1 Success BarackObama, 813286
2 Success JoeBiden, 939091
3 Success Dennis_Kucinich, 15022633
4 Success GovRichardson, 31428685
5 Success GovMikeHuckabee, 15416505
6 Success HillaryClinton, 1339835893
7 Success RonPaul, 287413569
8 Success SenJohnMcCain, 19394188
9 Success newtgingrich, 20713061
10 Success MikeGravel_US, 752638969095999489
11 Success loyaltoliberty, 19637821
12 Success MittRomney, 50055701
13 Success fredthompson, 2704951
14 Success SenChrisDodd, 16317406
15 Success RudyGiuliani, 770781940341288960
16 Success RalphNader, 77314692
17 Success ttancredo, 64534908
18 Success DrJillStein, 111216929
19 Success VirgilGoode, 605641396
20 Success GovGaryJohnson, 95713333
21 Success RickSantorum, 58379000
22 Success HillaryClinton, 1339835893
23 Success SenSanders, 29442313
24 Success realDonaldTrump, 25073877
25 Success JebBush, 113047940
26 Success TedCruz, 23022687
27 Success MarcoRubio, 15745368
28 Success JohnKasich, 18020081
29 Success SenBobCorker, 73303753
30 Success Mi

In [ ]:
# read in coordinates data
data = pd.read_csv("data/coordinates.csv", keep_default_na=False)
df = pd.DataFrame(data)
data_list = []

# flatten it so SQL can ingest it
for index, row in df.iterrows():

    row_list = df.loc[index, :].values.flatten().tolist()
    row_list[1] = str(row_list[1])
    data_list.append(row_list[1:])

In [109]:
# this ticker tracks the hits
i = 1

# this ticker tracks the batches
# we can really only hit this endpoint about 90 times per every 15 minutes
# this just tracks if we need to take a 15 min break or not
j = 1

for item in data_list:
    client.load_cookies('cookies.json')
    handle = item[3]
    user = client.get_user_by_screen_name(handle)
    created = int(str(user.created_at_datetime)[0:4])
    year = int(item[7])
    
    # if the twitter account was created before or during the election year
    # then it's True since we can pull tweets
    if created <= year:
        item[-1] = True
    
    # otherwise we can't pull tweets for this election for this politician
    else:
        item[-1] = False
    
    print(f"Batch {j}, item {i} : {item}")

    # cooldowns because of rate limits
    if i == 90 and j < 3:
        time.sleep(900)
        i = 1
        j += 1
    else:
        time.sleep(1)
        i += 1

# there is probably a better way to do this
# but it works...so I'm not gonna fix it

Batch 1, item 1 : [8132862008, 813286, 'Barack Obama', 'BarackObama', 3, 2, 'Democratic', 2008, 'USA', True]
Batch 1, item 2 : [9390912008, 939091, 'Joe Biden', 'JoeBiden', 3, 3, 'Democratic', 2008, 'USA', True]
Batch 1, item 3 : [150226332008, 15022633, 'Dennis Kucinich', 'Dennis_Kucinich', -2, -2, 'Democratic', 2008, 'USA', True]
Batch 1, item 4 : [314286852008, 31428685, 'Bill Richardson', 'GovRichardson', 4, 4, 'Democratic', 2008, 'USA', False]
Batch 1, item 5 : [154165052008, 15416505, 'Mike Huckabee', 'GovMikeHuckabee', 6, 6, 'Republican', 2008, 'USA', True]
Batch 1, item 6 : [13398358932008, 1339835893, 'Hillary Clinton', 'HillaryClinton', 4, 2, 'Democratic', 2008, 'USA', False]
Batch 1, item 7 : [2874135692008, 287413569, 'Ron Paul', 'RonPaul', 9, 1, 'Republican', 2008, 'USA', False]
Batch 1, item 8 : [193941882008, 19394188, 'John McCain', 'SenJohnMcCain', 6, 4, 'Republican', 2008, 'USA', False]
Batch 1, item 9 : [207130612008, 20713061, 'Newt Gingrich', 'newtgingrich', 8, 7, 

In [12]:
import numpy as np

# set numpy INT and BOOL to adapt to SQL acceptable data forms
sqlite3.register_adapter(np.int64, int)
sqlite3.register_adapter(np.bool_, str)

# set up SQL connection and cursor
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

In [6]:
# create table
c.execute("""CREATE TABLE coordinates (
          key STRING,
          twitter_user_id STRING,
          politician_name STRING,
          twitter_handle STRING,
          x_coordinate INTEGER,
          y_coordinate INTEGER,
          political_party STRING,
          election_year INTEGER,
          country STRING,
          twitter_active_during_election STRING
          )""")

In [14]:
# ingest the data FINALLY

c.executemany("INSERT INTO coordinates VALUES (?,?,?,?,?,?,?,?,?,?)", data_list)
conn.commit()